# Evaluating mean and variance of fleet catch

## Analytical expressions of interest

Expression for the mean of fleetwide quota: 

\begin{equation}-

## Initialization of parameters

In [ ]:
import scipy.integrate
from scipy.stats import norm
import matplotlib as plt 
import numpy as np 
from scipy import log 
import operator


In [96]:
# Initialize parameters 

# parameters for both models 
# Q = 45800000
# n = 100
Q = 400000
n = 15
# moment-match from Evelyn's manuscript, exp[F_t,i] = 22000
mu = 10
sig = 0.3

print find_expected(mu,sig*sig)
print find_variance(mu,sig*sig)
# first initialize them to have all the same mean, variance
# Keep in mind this parameterization is for the normal distribution describing log returns. 
# mean of returns = 22000 = e^(4+(1.5^2)/2) = 168
# variance of returns: e^()

fisher_mu_T = np.empty(n)
fisher_sigma_T = np.empty(n)
fisher_mu_T.fill(mu)
fisher_sigma_T.fill(sig)

fisher_mu_B = np.empty(n)
fisher_sigma_B = np.empty(n)
fisher_mu_B.fill(mu)
fisher_sigma_B.fill(sig)

IBQ = np.empty(n) 
IBQ.fill(Q / n)

23040.2968767
49992915.7608


## Function definitions: find parameters of distribution of fleetwide catch for (k) days

In [81]:
# Helper functions: find the moments of lognormal distribution, given its parameters mu and sigmasqd

def find_expected(mu, sigmasqd): 
    return np.exp(mu+sigmasqd/2)

def find_variance(mu, sigmasqd): 
    return np.exp(2*mu + sigmasqd)*(np.exp(sigmasqd) - 1)


# mu_B corresponds to the special case that k = 1
def find_sigmasqd_B(k):
    num = k * np.sum( (np.exp(2*fisher_mu_B+fisher_sigma_B)) * (np.exp(fisher_sigma_B*fisher_sigma_B) - 1) )
    denom = pow(k * np.sum( np.exp(fisher_mu_B+fisher_sigma_B/2) ),2) 
    return log(num / denom + 1)

def find_mu_B(sigmasqd_B, k): 
    return log( np.sum( np.exp(2*fisher_mu_B + fisher_sigma_B*fisher_sigma_B / 2) ) ) - sigmasqd_B / 2 

sigmasqd_B = find_sigmasqd_B(1)
mu_B = find_mu_B(sigmasqd_B,1)

NameError: name 'sigmasqd' is not defined

## Find fleet catch mean, variance under collective quota 

Numerically approximates the discrete expectation over stopping time k to compute: 


\begin{equation}
E[F_{T}] = \sum_k^{\infty}k (\sum_{i=1}^n e^{\mu_{i,T} + \frac{\sigma_i^2}{2} } )   \Phi \left[ \frac{log Q - \mu_{Z,B}}{\sigma_{Z,B}} \right]  \int_0^Q \Phi \left[ -\frac{log(Q-q)-\mu_B}{\sigma_B} \right] \frac{1}{q \sigma_{Z,B} \sqrt{2 \pi}} exp \left[ - \frac{(log (q)- \mu_{Z,B})^2}{2 \sigma_{Z,B}^2}  \right] dq 
\end{equation} 

\begin{align}
 Var[F_{T}] =  \sum_k^{\infty} k \sum_{i=1}^n e^{2 \mu_{i,T} + \sigma_{i,T}^2} (e^{\sigma_i^2} -1)   \Phi \left[ \frac{log Q - \mu_{Z,B}}{\sigma_{Z,B}} \right]  \int_0^Q \Phi \left[ -\frac{log(Q-q)-\mu_B}{\sigma_B} \right] \frac{1}{q \sigma_{Z,B} \sqrt{2 \pi}} exp \left[ - \frac{(log (q)- \mu_{Z,B})^2}{2 \sigma_{Z,B}^2}  \right] dq 
\end{align} 

In [98]:

# fleetwide catch: 
# start from k = 0 
def find_fleet_catch(fisher_mu_B, fisher_sigma_B, mu_B, sigmasqd_B, fisher_mu_T, fisher_sigma_T):
    #helper function for the probability of the fleetwide catch at this time k bringing us above quota
    def integrand(q): 
        return norm.cdf( (log(Q) - mu_Z_B)/ np.sqrt(sigmasqd_Z_B) ) * norm.cdf(-1 * (log(Q - q) - mu_B)/ np.sqrt(sigmasqd_B) ) * (1/ (q * np.sqrt(sigmasqd_Z_B)*np.sqrt(2*np.pi))) * np.exp(-1* pow((log(q) - mu_Z_B),2)/(2*sigmasqd_Z_B))
                       
    # keep track of some delta, terminate when some precision is met 
    epsilon = 0.000000001 
    k = 2
    prev = [0,0]
    result = [0,0]
    delta = 1000
    while (delta > epsilon) and (k < 10000):
        # Find parameters of the distribution of the fleetwide catch up until time k-1
        print k-1
        sigmasqd_Z_B = find_sigmasqd_B(k-1)
        mu_Z_B = find_mu_B(sigmasqd_Z_B, k-1)
        
        print(mu_Z_B)
        print(sigmasqd_Z_B)
        
        print find_expected(mu_Z_B, sigmasqd_Z_B)
        print find_variance(mu_Z_B, sigmasqd_Z_B)
        
        single_catch = np.sum( find_expected(fisher_mu_T, fisher_sigma_T*fisher_sigma_T) )
        single_var = np.sum( find_variance(fisher_mu_T, fisher_sigma_T*fisher_sigma_T) ) 

        # compute probability of the fleetwide catch at this time k bringing us above quota by integrating over possible values of F_{k-1,B}
        common = scipy.integrate.quad( integrand , 0, Q)[0]
        this = [k * single_catch * common, k*k  * single_var * common] 
        
        print scipy.integrate.quad( integrand , 0, Q)
        
        delta = max( [ abs(x) for x in map(operator.sub, this, prev) ] ) 
        prev = this
        result = map(operator.add, result, this)
        print "________________________________"
        k += 1
        if (k % 100 == 0): 
            print str(k) + ", this step: " + str(this) + ", result (mean, variance): " + str(result) + ", delta: " + str(delta)
        
    return result

find_fleet_catch(fisher_mu_B, fisher_sigma_B, mu_B, sigmasqd_B, fisher_mu_T, fisher_sigma_T)

1
22.7499208715
0.00625865924907
7612444665.85
3.63822341163e+17
(0.0, 0.0)
________________________________


[0.0, 0.0]

In [ ]:
## Find fleet catch mean, variance under collective quota 

Numerically evaluates the discrete expectations to compute: 


\begin{equation}
E[F_{T}] = \sum_k^{\infty}k (\sum_{i=1}^n e^{\mu_{i,T} + \frac{\sigma_i^2}{2} } )   \Phi \left[ \frac{log Q - \mu_{Z,B}}{\sigma_{Z,B}} \right]  \int_0^Q \Phi \left[ -\frac{log(Q-q)-\mu_B}{\sigma_B} \right] \frac{1}{q \sigma_{Z,B} \sqrt{2 \pi}} exp \left[ - \frac{(log (q)- \mu_{Z,B})^2}{2 \sigma_{Z,B}^2}  \right] dq 
\end{equation} 

\begin{align}
 Var[F_{T}] =  \sum_k^{\infty} k \sum_{i=1}^n e^{2 \mu_{i,T} + \sigma_{i,T}^2} (e^{\sigma_i^2} -1)   \Phi \left[ \frac{log Q - \mu_{Z,B}}{\sigma_{Z,B}} \right]  \int_0^Q \Phi \left[ -\frac{log(Q-q)-\mu_B}{\sigma_B} \right] \frac{1}{q \sigma_{Z,B} \sqrt{2 \pi}} exp \left[ - \frac{(log (q)- \mu_{Z,B})^2}{2 \sigma_{Z,B}^2}  \right] dq 
\end{align} 

In [ ]:
# find fleet catch and variance for individual quotas
def find_fleet_catch_IBQ(fisher_mu_B, fisher_sigma_B, mu_B, sigmasqd_B, fisher_mu_T, fisher_sigma_T, IBQ):
    
    def integrand(q): 
        # assumes IBQ same for all fishers
        return norm.cdf( (log(Q/n) - mu_Z_B)/ np.sqrt(sigmasqd_Z_B) ) * norm.cdf(-1 * (log(Q/n - q) - mu_B)/ sigmasqd_B ) * (1/ (q * np.sqrt(sigmasqd_Z_B)*np.sqrt(2*np.pi))) * np.exp(-1* pow((log(q) - mu_Z_B),2)/(2*sigmasqd_Z_B))
    def compute_integral(ibq):
        return scipy.integrate.quad( integrand , 0, ibq)[0]
    integral_over_IBQ = np.vectorize(compute_integral)
     # keep track of some delta, terminate when some precision is met 
    epsilon = 0.000000001 
    k = 2
    prev = [0,0]
    result = [0,0]
    delta = 1000
    while (delta > epsilon) and (k < 10000):
        
        sigmasqd_Z_B = find_sigmasqd_B(k-1)
        mu_Z_B = find_mu_B(sigmasqd_Z_B, k-1)
        
        catch = np.exp(fisher_mu_T+fisher_sigma_T*fisher_sigma_T/2) 
        var = np.exp(2*fisher_mu_T + fisher_sigma_T*fisher_sigma_T) * (np.exp(fisher_sigma_T) - 1) 
        
        # compute this iteration with k 
        # ~FIXME this hack uses the simplification that all fishers have same IBQ; not great for generalizability
        common = integral_over_IBQ(IBQ)
        this = [ k *np.sum( catch * common ), k*k  * np.sum(single_var * common) ] 
        
        print common
        
        delta = max( [ abs(x) for x in map(operator.sub, this, prev) ] ) 
        prev = this
        result = map(operator.add, result, this)
        
        k += 1
        if (k % 100 == 0): 
            print str(k) + ", this step: " + str(this) + ", result (mean, variance): " + str(result) + ", delta: " + str(delta)
        
    return result